# Day-1

# Understand business problem
 # 1.Objective:
 Predict the next month’s product sales based on historical sales data.

Evaluate predictions using MAPE (Mean Absolute Percentage Error) and RMSE (Root Mean Squared Error).
# 2. Collect Dataset
In this step, we load the historical retail sales dataset.  
The dataset contains monthly sales data for different products over several years.  
We will use this data to analyze sales trends and forecast next month's sales.


<!-- Objective: -->

In [80]:
import pandas as pd
import numpy as np

In [81]:
df = pd.read_csv("Walmart.csv")

In [82]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [83]:
df.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')

In [84]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)


In [85]:
df = df.sort_values('Date')
df.head()


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1287,10,2010-02-05,2193048.75,0,54.34,2.962,126.442065,9.765
5148,37,2010-02-05,536006.73,0,45.97,2.572,209.852966,8.554
2288,17,2010-02-05,789036.02,0,23.11,2.666,126.442065,6.548
4147,30,2010-02-05,465108.52,0,39.05,2.572,210.752605,8.324


In [86]:
df.shape

(6435, 8)

# Day-2

In [87]:
# Step 1
df.isnull().sum()

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [88]:
df.columns

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')

In [89]:
df = df.sort_values(["Store", "Date"])
df.head()


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


In [92]:
df['Lag_1'] = df.groupby('Store')['Weekly_Sales'].shift(1)
df['Lag_2'] = df.groupby('Store')['Weekly_Sales'].shift(2)
df['Lag_3'] = df.groupby('Store')['Weekly_Sales'].shift(3)

df[['Lag_1','Lag_2','Lag_3']] = df[['Lag_1','Lag_2','Lag_3']].fillna(0)
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Lag_1,Lag_2,Lag_3
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,0.00,0.00,0.00
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,1643690.90,0.00,0.00
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,1641957.44,1643690.90,0.00
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,1611968.17,1641957.44,1643690.90
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,1409727.59,1611968.17,1641957.44
